In [1]:
import aurel
import numpy as np
from aurel.solutions import Conformally_flat as sol
from scipy.interpolate import RegularGridInterpolator
import spinsfast

In [ ]:
L = 3
N = 72
param = { 'Nx': N, 'Ny': N, 'Nz': N, 
          'xmin': -L/2, 'ymin': -L/2, 'zmin': -L/2, 
          'dx': L/N, 'dy': L/N, 'dz': L/N}
t = 0.0
fd = aurel.FiniteDifference(param)
rel = aurel.AurelCore(fd, verbose=False)
rel.data = sol.data(t, fd.x, fd.y, fd.z)
rel.freeze_data()

4th order finite difference schemes are defined


<IPython.core.display.Latex object>

In [16]:
def Psi4_lm(r_sphere):
    lmax = 8
    Ntheta = 2*lmax + 1
    Nphi = 2*lmax + 1

    theta = np.linspace(0, np.pi, Ntheta)
    phi = np.linspace(0, 2*np.pi, Nphi, endpoint=False)
    theta_sphere, phi_sphere = np.meshgrid(theta, phi, indexing='ij')

    x_sphere = r_sphere * np.sin(theta_sphere) * np.cos(phi_sphere)
    y_sphere = r_sphere * np.sin(theta_sphere) * np.sin(phi_sphere)
    z_sphere = r_sphere * np.cos(theta_sphere)
    points_sphere = np.stack(
        (x_sphere.flatten(), y_sphere.flatten(), z_sphere.flatten()), axis=-1)

    # Psi4 on a sphere
    # sphere around the origin with radius 1
    Psi4 = rel['Weyl_Psi'][4]
    interp_real = RegularGridInterpolator(
        (fd.xarray, fd.yarray, fd.zarray), np.real(Psi4), 
        bounds_error=False, fill_value=0)
    interp_imag = RegularGridInterpolator(
        (fd.xarray, fd.yarray, fd.zarray), np.imag(Psi4), 
        bounds_error=False, fill_value=0)
    psi4_sphere = interp_real(points_sphere) + 1j * interp_imag(points_sphere)
    psi4_sphere = psi4_sphere.reshape(Ntheta, Nphi)

    # lm mode of Psi4
    alm = spinsfast.map2salm(theta_sphere, -2, lmax)
    lm_dict = {}
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            lm_dict[l, m] = alm[l**2 + m + l]
    return lm_dict

In [19]:
Psi4_lm(1)

{(0, 0): np.complex128(-0+0j),
 (1, -1): np.complex128(0j),
 (1, 0): np.complex128(-0+0j),
 (1, 1): np.complex128(-0j),
 (2, -2): np.complex128(2.80553942438068e-17+0j),
 (2, -1): np.complex128(8.546044971242637e-17-3.28919271857445e-33j),
 (2, 0): np.complex128(5.083164752420469-6.225081443590376e-16j),
 (2, 1): np.complex128(6.730885007874829e-17-1.0420684579357859e-32j),
 (2, 2): np.complex128(2.6485213187778702e-17-6.487006311029691e-33j),
 (3, -3): np.complex128(7.361752683005684e-18-1.3096990393221705e-33j),
 (3, -2): np.complex128(2.9131792499469056e-17+0j),
 (3, -1): np.complex128(-1.9636941574675802e-17+1.6569689295569014e-33j),
 (3, 0): np.complex128(-1.8912420850118556+2.3161035658225354e-16j),
 (3, 1): np.complex128(-1.432159185088158e-17+2.1762806954981552e-33j),
 (3, 2): np.complex128(-4.2492517690426484e-17+1.0407665155458615e-32j),
 (3, 3): np.complex128(-2.9729471923690708e-18+2.670678948082261e-33j),
 (4, -4): np.complex128(1.4428388171575308e-17+1.7669679391175233e-3